In [1]:
import sys 
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.pose import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.protocols.minimization_packing import *
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose, glycosylate_pose_by_file
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.protocols.rosetta_scripts import *

## Personal Imports|
import basePeptidePrep
#import addBaseSugarAndEnzyme

## Pyrosetta Initialization
pyrosetta.init('-include_sugars -write_pdb_link_records -ex1 -ex2 -alternate_3_letter_codes pdb_sugar -auto_detect_glycan_connections -min_bond_length 1.1 -max_bond_length 1.5 -ignore_zero_occupancy false -ignore_unrecognized_res false')

## Command Line Arguments 
'''
Command Line Arguments:
# For the base peptide 
- Sequence of the base peptide (2-3 AA long peptides are ideal) : base_seq
- Position of the sequence containing THR that will be glycosylated : base_pos
- the sugar that will be glycosylated on the base peptide : base_sugar

# input pdb files
- The enzyme with donor peptide 

'''
base_seq = "STP"
base_position = 2
base_sugar = "core1"
input_enzyme_file = "/home/souvadra/myGitFolders/Glycosyltransferase/glycan_sampler_pipeline/output_3OTK-closed-S217C/3OTK-closed-monomer-alpha-GlcNAc-S217C_0005.pdb"
## Prepare the base peptide 
base_pose = basePeptidePrep.basePeptideBuild(base_seq, base_position, base_sugar)
print(base_pose)

## Addition of peptide and sugar motif 
enzyme_pose = pose_from_pdb(input_enzyme_file)


PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.40+release.f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 2020-10-02T11:47:11] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r267 2020.40+release.f01fa77 f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 http://www.pyrosetta.org 2020-10-02T11:47:11
core.init: {0} command: PyRosetta -include_sugars -write_pdb_link_records -ex1 -ex2 -alternate_3_letter_codes pdb_sugar -auto_detect_glycan_connections -min_bond_length 1.1 -max_bond_length 1.5 -ignore_zero_occupancy false -ignore_unrecognized_res false -database /home/souvadra/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator:

protocols.carbohydrates.GlycanSampler: {0} Randomizing glycan torsions
protocols.carbohydrates.GlycanSampler: {0} Modeling 2 glycan residues
protocols.carbohydrates.GlycanSampler: {0} Setting up Tree-based packing.
protocols.carbohydrates.GlycanSampler: {0} Initialized
protocols.carbohydrates.GlycanSampler: {0} starting energy: 1004.11
protocols.carbohydrates.GlycanSampler: {0} Total Rounds = 200 ( 2 glycans * 100 )
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing from carbohydrate root: 5
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing 1 residues
protocols.carbohydrates.LinkageConformerMover: {0} Sampling beta-D-Galp-(?->3)-alpha-D-GalpNAc- linkage
protocols.carbohydrates.LinkageConformerMover: {0} Sampling conformer 1 which has a population of 1
protocols.carbohydrates.LinkageConformerMover: {0} Complete
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 243 rotamers at 1 positions.
core.pack.pack_rotamers: {0} Requ

protocols.carbohydrates.LinkageConformerMover: {0} Complete
protocols.carbohydrates.GlycanSampler: {0} Round: 60
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSet_: {0} [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 2 THR:O-conjugated
core.pack.pack_rotamers: {0} built 406 rotamers at 3 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last m

protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing from carbohydrate root: 5
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing 1 residues
protocols.carbohydrates.GlycanSampler: {0} Round: 120
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing from carbohydrate root: 4
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing 2 residues
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 405 rotamers at 2 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
protocols.carbohyd

protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing from carbohydrate root: 4
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing 2 residues
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing from carbohydrate root: 4
protocols.carbohydrates.GlycanTreeMinMover: {0} Minimizing 2 residues
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: {0} Last mover failed.  Continueing!
protocols.carbohydrates.GlycanSampler: {0} Round: 190
protocols.carbohydrates.LinkageConformerMover: {0} Sampling be

protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.

protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: {0} [ WARNING ] no movable positions in ShearMover!
protocols.

In [2]:
enzyme_pose.append_pose_by_jump(base_pose, 369)

#sugar_and_enzyme_pose = addBaseSugarAndEnzyme(base_pose, enzyme_pose, "constraints")

core.conformation.carbohydrates.GlycanTreeSet: {0} Setting up Glycan Trees
core.conformation.carbohydrates.GlycanTreeSet: {0} Found 2 glycan trees.


In [ ]:
print(enzyme_pose.total_residue())

In [ ]:
print(base_pose)
print(enzyme_pose)

In [ ]:
base_pose.dump_pdb("merged.pdb")

In [28]:
print(enzyme_pose.fold_tree())
print(enzyme_pose.fold_tree().size())

FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 370 371 -1  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 373 375 -2    EDGE 375 376 -1 
9


In [ ]:
ft = pyrosetta.FoldTree()
ft.reorder(1)

In [18]:
first_residue_enzyme = 1
last_residue_enzyme = 369
anchor_residue_enzyme = 130
anchor_residue_substrate = 373
first_residue_substrate = 372
last_residue_substrate = 374
first_donor = 370
last_donor = 371

ft_docking = pyrosetta.FoldTree()

'''
for downElem in range(first_residue_enzyme,anchor_residue_enzyme):
    downElem = anchor_residue_enzyme  - downElem 
    ft_docking.add_edge(downElem + 1, downElem, -1) 
    # making a bond chain of 130-129, 129-128 ... 
for upElem in range(anchor_residue_enzyme, last_residue_enzyme): 
    ft_docking.add_edge(upElem, upElem+1, -1)
    # making a bond chain of 130-131, 131-132 ... 
'''
ft_docking.add_edge(anchor_residue_enzyme,1,-1)
ft_docking.add_edge(anchor_residue_enzyme,last_residue_enzyme,-1)

ft_docking.add_edge(anchor_residue_enzyme, anchor_residue_substrate, 1) 
ft_docking.add_edge(anchor_residue_enzyme, first_donor, 2)
        
ft_docking.add_edge(anchor_residue_substrate, first_residue_substrate, -1) # downstream base peptide 
ft_docking.add_edge(anchor_residue_substrate, last_residue_substrate, -1) # upstream base peptide 
ft_docking.add_edge(first_donor,last_donor,-1) # the donor UDP-GlcNAc backbone

# setup branches 
chemical_edges = enzyme_pose.fold_tree().get_chemical_edges() # connections to the glycan residues from anchor out 
for chemical in range(1, len(chemical_edges) + 1):
    ft_docking.add_edge(chemical_edges[chemical].start(), chemical_edges[chemical].stop(), -2)

#ft_docking.add_edge(anchor_residue_substrate, first_donor, 3)

ft_docking.add_edge(375,376,-1) # Adding the covalent bond between the two sugars of core 1 
print(ft_docking)

FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 370 371 -1  EDGE 373 375 -2    EDGE 373 370 3  EDGE 375 376 -1 


In [19]:
ft_docking.check_fold_tree()

core.kinematics.FoldTree: {0} [ ERROR ] Bad fold tree at edge  EDGE 373 370 3
core.kinematics.FoldTree: {0} [ ERROR ] Stop residue 370 has already been built.
core.kinematics.FoldTree: {0} [ ERROR ] FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 370 371 -1  EDGE 373 375 -2    EDGE 373 370 3  EDGE 375 376 -1


False

In [17]:
print(enzyme_pose.fold_tree())
enzyme_pose.fold_tree(ft_docking)
print(enzyme_pose.fold_tree())

FOLD_TREE  EDGE 1 369 -1  EDGE 1 370 1  EDGE 370 371 -1  EDGE 369 372 2  EDGE 372 374 -1  EDGE 373 375 -2  OG1  C1   EDGE 375 376 -1 
FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 370 371 -1  EDGE 373 375 -2    EDGE 375 376 -1 


In [16]:
print(ft_docking)

FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 370 371 -1  EDGE 373 375 -2    EDGE 375 376 -1 


In [10]:
print(enzyme_pose.fold_tree())

FOLD_TREE  EDGE 1 369 -1  EDGE 1 370 1  EDGE 370 371 -1  EDGE 369 372 2  EDGE 372 374 -1  EDGE 373 375 -2  OG1  C1   EDGE 375 376 -1 


In [13]:
ft = pyrosetta.FoldTree()
ft.add_edge(130,1,-1)
ft.add_edge(130,369,-1)
ft.add_edge(130,373,1)
ft.add_edge(130,370,2)
ft.add_edge(370,371,-1)
ft.add_edge(373,372,-1)
ft.add_edge(373,374,-1)
ft.add_edge(373,375,-2)
ft.add_edge(375,376,-1)

In [15]:
enzyme_pose.fold_tree(ft)
print(enzyme_pose.fold_tree())

FOLD_TREE  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 130 373 1  EDGE 130 370 2  EDGE 370 371 -1  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 373 375 -2    EDGE 375 376 -1 


In [5]:
print(ft)

FOLD_TREE  EDGE 130 373 1  EDGE 130 370 2  EDGE 370 373 3  EDGE 130 1 -1  EDGE 130 369 -1  EDGE 370 371 -1  EDGE 373 372 -1  EDGE 373 374 -1  EDGE 375 376 -1  EDGE 373 375 -2   


In [16]:
ft.check_fold_tree()
ft.reorder(130)

True

In [ ]:
ft2 = pyrosetta.FoldTree()
ft2.reorder(130)
enzyme_pose.fold_tree(ft2)

In [ ]:
minimizer =  pyrosetta.rosetta.protocols.minimization_packing.MinMover()

In [ ]:
minimizer.cartesian(True)

In [ ]:
minimizer.min_type("lbfgs_armijo_nonmonotone")

In [ ]:
minimizer.constraintscst_file("constraints_file.cst")
minimizer.score_function(sfxn)
minimizer.apply(enzyme_pose)

In [ ]:
sfxn =  ref2015_cart.wts

In [ ]:
pyrosetta.rosetta.core.scoring.ScoreFunction("ref2015_cart.wts")

In [ ]:
chemical_edges = enzyme_pose.fold_tree().get_chemical_edges() # connections to the glycan residues from anchor out 

In [ ]:
len(chemical_edges)

In [ ]:
chemical_edges

In [ ]:
chemical_edges

In [6]:
init("-constraints:cst_fa_file " + constraints_file)

NameError: name 'constraints_file' is not defined

In [5]:
sfxn = pyrosetta.create_score_function("ref2015_cart.wts") #get_fa_scorefxn()
sfxn.set_weight(atom_pair_constraint, 1.0)
add_fa_constraints_from_cmdline(enzyme_pose, sfxn)
print(sfxn.show(enzyme_pose))

minimizer =  pyrosetta.rosetta.protocols.minimization_packing.MinMover()
minimizer.cartesian(True)
minimizer.tolerance(0.001)
minimizer.min_type("lbfgs_armijo_nonmonotone")
minimizer.score_function(sfxn)
minimizer.apply(enzyme_pose)

core.scoring.CartesianBondedEnergy: {0} Initializing IdealParametersDatabase with default Ks=300 , 80 , 80 , 10 , 80
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-lengths.txt
core.scoring.CartesianBondedEnergy: {0} Read 759 bb-independent lengths.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-angles.txt
core.scoring.CartesianBondedEnergy: {0} Read 1434 bb-independent angles.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-torsions.txt
core.scoring.CartesianBondedEnergy: {0} Read 1 bb-independent torsions.
basic.io.database: {0} Database file opened: scoring/score_functions/bondlength_bondangle/default-improper.txt
core.scoring.CartesianBondedEnergy: {0} Read 529 bb-independent improper tors.


NameError: name 'add_fa_constraints_from_cmdline' is not defined

In [19]:
print(enzyme_pose.pdb_info().pdb2pose('A',358))

303


In [21]:
print(enzyme_pose.residue(375))

Residue 375: ->3)-alpha-D-Galp:2-AcNH (Gal, Z):
Base: ->3)-alpha-D-Galp
 Properties: POLYMER CARBOHYDRATE POLAR CYCLIC HEXOSE ALDOSE D_SUGAR PYRANOSE ALPHA_SUGAR C2_MODIFIED ACETYLAMINO_SUGAR
 Variant types: C2_ACETYLAMINO_SUGAR
 Main-chain atoms:  C1   C2   C3   O3 
 Backbone atoms:    C1   C2   C3   O3   C4   C5   O5   VO5  VC1  H1   H2   H3   H4   H5 
 Ring atoms:    C1   C2   C3   C4   C5   O5 
 Side-chain atoms:  O1   O2   O4   C6   O6   HO1  HO3  HO2  N2   CN2 CAN2 OCN2  HO4 1H6  2H6   HO6  HN2 1HC2 2HC2 3HC2
Carbohydrate Properties for this Residue:
 Basic Name: galactosamine
 IUPAC Name: ->3)-alpha-2-(N-acetylamino)-2-deoxy-D-galactopyranosyl
 Abbreviation: ->3)-alpha-D-GalpNAc-
 Classification: aldohexose
 Stereochemistry: D
 Ring Form: pyranose
 Anomeric Form: alpha
 Modifications: 
  acetylamino sugar
 Polymeric Information:
  Reducing?: no
  Main chain connection: (_->3)
  Branch connections: none
Ring Conformer: 4C1 (chair): C-P parameters (q, phi, theta): 0.55, 180, 0; nu